For analysis in NMF and representation in figures, electrodes must meet one of two criteria:
1. A p-value of <= 0.05, as determined by the bootstrapping method in `results/stats/bootstrap_electrode_significance.ipynb`. Either a `spkr_p` or `mic_p` under the threshold is sufficient.
2. An r-value of >= 0.1 in mTRF model 1 ("full" model), calculated in `ecog/strf/ecog_strf.ipynb`

There are also several exclusion criteria:
1. The electrode is outside the brain, or next to an outside-the-brain electrode and has a similar waveform to the one outside the brain. These were manually entered by GLK and stored in `./manually_excluded_electrodes.csv`.
2. The waveform of the ERP is reduced to noise when a bipolar reference is used instead of the regular common average reference. These were manually entered by GLK and stored in `./manually_excluded_electrodes_bipolar.csv`.

This notebook generates two csvs, `./all_included_electrodes.csv` and `./all_excluded_electrodes.csv`.

In [2]:
import mne
import os
import numpy as np
import pandas as pd
from img_pipe import img_pipe
import warnings

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import rcParams as rc
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
import seaborn as sns
%matplotlib inline

In [ ]:
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [1]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

NameError: name 'os' is not defined

In [ ]:
# get mtrf rvals
rdf = pd.read_csv.os.path.join(git_path,"analysis","mtrf","results.csv")
rdf = rdf.loc[rdf['model']=='model1']
rvals = dict()
for s in subjs:
    rvals[s] = dict()
    ch_names = list(rdf.loc[rdf['subject']==s]['channel'].values)
    for ch in ch_names:
        rvals[s][ch] = rdf.loc[(rdf['subject']==s)&(rdf['channel']==ch)]['r_value'].values[0]

In [ ]:
# get pvals
pdf = pd.read_csv(os.path.join(git_path,"stats","bootstraps","seeg_elec_significance_16_subjs_1000_boots.csv"))
pvals = dict()
for s in subjs:
    pvals[s] = dict()
    ch_names = list(pdf.loc[pdf['subj']==s]['ch_name'].values)
    for ch in ch_names:
        if ch != "EKG1":
            pvals[s][ch] = dict()
            pvals[s][ch]['spkr'] = pdf.loc[(pdf['subj']==s)&(pdf['ch_name']==ch)]['spkr_p'].values[0]
            pvals[s][ch]['mic'] = pdf.loc[(pdf['subj']==s)&(pdf['ch_name']==ch)]['mic_p'].values[0]

In [ ]:
# get outside the brain
outside_brain = dict()
for s in subjs:
    if s not in no_anat:
        in_bolt_fpath = os.path.join(data_path,f"{s}_complete","elecs",f"{s}_IN_BOLT.txt")
        elecs_in_bolt = np.loadtxt(in_bolt_fpath, dtype=str, skiprows=1)
        if len(elecs_in_bolt.shape) > 0:
            if elecs_in_bolt.shape[0] != 0:
                elecs_in_bolt = list(elecs_in_bolt)
            else:
                elecs_in_bolt = []
        else:
            elecs_in_bolt = [str(elecs_in_bolt)]
        outside_brain[s] = elecs_in_bolt
    else:
        outside_brain[s] = []

In [ ]:
# get manually excluded
manual_excludes = dict()
excl_df = pd.read_csv(os.path.join(git_path,"results","ecog_paper","manually_excluded_electrodes.csv"))
for s in subjs:
    manual_excludes[s] = list(excl_df.loc[excl_df['subject']==s]['channel'].values)

In [ ]:
# get manually excluded (bipolar ref)
manual_excludes_bp = dict()
excl_df = pd.read_csv(os.path.join(git_path,"analysis","manually_excluded_electrodes_bipolar.csv"))
for s in subjs:
    manual_excludes_bp[s] = list(excl_df.loc[excl_df['subject']==s]['channel'].values)

In [ ]:
# Evaluate inclusion, subject-by-subject
include_chs = dict(); include_pvals, include_r = dict(), dict()
exclude_chs = dict()
for s in subjs:
    include_chs[s] = []; include_pvals[s], include_r[s] = [], []
    exclude_chs[s] = []
    ch_names = list(pvals[s].keys())
    for ch in ch_names:
        spkr_p = pvals[s][ch]['spkr']
        mic_p = pvals[s][ch]['mic']
        r = rvals[s][ch]
        if ch not in manual_excludes[s] and ch not in manual_excludes_bp[s] and ch not in outside_brain[s]:
            if spkr_p < 0.05 or mic_p < 0.05:
                include_chs[s].append(ch)
                include_pvals[s].append(ch)
            elif r >= 0.1:
                include_chs[s].append(ch)
            if r >= 0.1:
                include_r[s].append(ch)
            else:
                exclude_chs[s].append(ch)
        else:
            exclude_chs[s].append(ch)
    print(f"{s}: Including {len(include_chs[s])} channels & excluding {len(exclude_chs[s])} channels.")
    print(f"{s}: {len(include_pvals[s])} channels with significant pvals and {len(include_r[s])} channels with good TRF fit.")
    print("")

In [ ]:
# save to txtfiles
include_txt = [["subject","channel"]]
exclude_txt = [["subject","channel"]]
for s in subjs:
    for ch in include_chs[s]:
        include_txt.append([s, ch])
    for ch in exclude_chs[s]:
        exclude_txt.append([s, ch])
np.savetxt(os.path.join(git_path,"analysis", "all_included_electrodes.csv"),
           np.array(include_txt), delimiter=",", fmt="%s")
np.savetxt(os.path.join(git_path,"analysis", "all_excluded_electrodes.csv"),
           np.array(exclude_txt), delimiter=",", fmt="%s")